In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 27 16:35:06 2018

@author: Bart van Voorden
"""
import sys, os
sys.path.insert(0, 'C:\\Users\\Bart van Voorden\\Dropbox\\PhD 2016\\Python PhD')
sys.path.insert(0, 'C:\\Users\\bart_\\Dropbox\\PhD 2016\\Python PhD')
import matplotlib_funcs as plot

import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import seaborn, pickle, datetime
seaborn.set("notebook",font_scale=1.45)

sqrt, e, pi = np.sqrt, np.e, np.pi
cos, sin = np.cos, np.sin

def c_d1(lab0, theta):
    return 2*(1+lab0**2 -2*lab0*cos(theta))

def c_d2(lab0, theta):
    return 2*(1+lab0**2 +2*lab0*cos(theta))

def c_h(k, lab0, theta):
    return 2*cos(k/2) * (1-lab0**2) - 4* sin(theta)*sin(k/2)*lab0

def c_ht(k, lab0, theta):
    return 2*sin(k/2) * (1-lab0**2) + 4* sin(theta)*cos(k/2)*lab0

def create_hamiltonian(N, k, lab0, theta):
    d1, d2 = c_d1(lab0, theta), c_d2(lab0, theta)
    d = d1+d2
    h, ht = c_h(k, lab0, theta), c_ht(k, lab0, theta)
    
    ham = np.diag([d2] + [d]*(N-2) + [d1])
    ham += np.diag([h]*(N-1),1)
    ham += np.diag([h]*(N-1),-1)
    ham[N-1,0] = ht
    ham[0,N-1] = ht
    
    return ham

def calc_eigs_point(N=3, k=2*pi/3, lab0=1, theta=0.1):
    Ham = create_hamiltonian(N, k, lab0, theta)
    eigvals, eigvecs = la.eig(Ham)
    idx = eigvals.argsort()[::1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:,idx]
    
    return eigvals.real, eigvecs

def calc_eigs_alltheta(N=3, k=2*pi/3, lab0=1, numtheta=101, plot_vals=True):
    eigvals, eigvecs = [], []
    
    thetarange = np.linspace(0, 2*pi, numtheta)
    for theta in thetarange:
        vals, vecs = calc_eigs_point(N, k, lab0, theta)
        eigvals.append(vals)
        eigvecs.append(vecs)
    
    eigvals, eigvecs = np.array(eigvals), np.array(eigvecs)
    if plot_vals:
        kval = int(N * k / (2*pi))
        names_axes = (r"$\theta$ (units of $\pi$)", r"$E$") 
        title = r"$N=${}, $k=${}$\pi/${}, $\lambda=${}$e^{{i \theta}}$".format(N,2* kval,N, lab0)
        plot.multiline(thetarange/pi, eigvals.transpose(), title=title, names_axes=names_axes )
        
    return np.array(eigvals), np.array(eigvecs)
    
    
def calc_eigs_allk(N=3, lab0=1, numtheta=101, plot_indi=False):
    eigvals = []
    eigvecs = []
    labels = []
    
    kvals = np.linspace(0,2*pi,N, endpoint=False)
    kvals[kvals>pi] -= 2*pi
    kvals.sort()
    
    for k in kvals:
        vals, vecs = calc_eigs_alltheta(N, k, lab0, numtheta, plot_indi)
        eigvals.append(vals)
        eigvecs.append(vecs)
        kval = int(N * k / (2*pi))
        labels.append(r'$k=${}$\pi/${}'.format(2* kval,N))
            
    return np.array(eigvals), np.array(eigvecs), labels
       

def plot_eigvals_allk(N=3, lab0=1, numtheta=101, plot_indi=False, returnvals=False):
    eigvals, eigvecs, labels = calc_eigs_allk(N, lab0, numtheta, plot_indi)
    
    fig, ax1 = plt.subplots()
    for j in range(N):
        dataylist = (eigvals[j]).transpose()
        c = seaborn.color_palette()[j%6]
        for i in range(len(dataylist)):
            ax1.plot(np.linspace(0,2, numtheta),  dataylist[i], label=labels[j], c=c)
    
    legend = plt.legend(frameon=True)
    frame = legend.get_frame()
    frame.set_facecolor('white') 
    
    ax1.set_xlabel(r"$\theta$ (units of $\pi$)")
    ax1.set_ylabel(r"$E$")
    plt.title(r"$N=${}, $\lambda=${}$e^{{i \theta}}$".format(N, lab0))

    if returnvals:
        return eigvals, eigvecs, labels
    
def plot_eigvecs_1k(N=3, k=2*pi/3, elevel=0, lab0=1, numtheta=101, plot_indi=False, returnvals=False):
    eigvals, eigvecs = calc_eigs_alltheta(N, k, lab0, numtheta, plot_indi)
    labels = ["L={}".format(i) for i in range(1,2*N,2)]

    plotvec = eigvecs[1:-1,:,elevel]
    plot.scatter(np.linspace(0,2,numtheta)[1:-1], abs(plotvec)**2, legend=True, labels=labels)    


In [2]:
plt.rcParams["figure.figsize"] = (12,8)

In [3]:
def plot_eigs_alltheta(N, p, q):
    lab0=1
    numtheta=101
    k=p*pi/q
    eigvals, eigvecs = [], []
    
    thetarange = np.linspace(0, 2*pi, numtheta)
    for theta in thetarange:
        vals, vecs = calc_eigs_point(N, k, lab0, theta)
        eigvals.append(vals)
        eigvecs.append(vecs)
    
    eigvals, eigvecs = np.array(eigvals), np.array(eigvecs)
    
    names_axes = (r"$\theta$ (units of $\pi$)", r"$E$") 
    title = r"$N=${}, $k=${}$\pi/${}, $\lambda=${}$e^{{i \theta}}$".format(N,p,q, lab0)
    plot.multiline(thetarange/pi, eigvals.transpose(), title=title, names_axes=names_axes )

from ipywidgets import interactive
interactive_plot = interactive(plot_eigs_alltheta, N=(3, 10, 1), p=(-10,10,1),q=(3,10,1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=6, description='N', max=10, min=3), IntSlider(value=0, description='p', max=10, min=-10), IntSlider(value=6, description='q', max=10, min=3), Output()), _dom_classes=('widget-interact',))

In [3]:
def plot_eigvector(N, p, q, t):
    theta = t*pi
    lab0=1
    k=p*pi/q
    numtheta=101
    
    fig, axes = plt.subplots(1,2, figsize=(16,7))
    ax1,ax2 = axes
    suptitle = r"$N=${}, $k=${}$\pi/${}, $\lambda_0=${}, $\theta=${}$\pi$".format(N,p,q, lab0, t) 
    fig.suptitle(suptitle, size=18)
    
    vals, vecs = calc_eigs_point(N, k, lab0, theta)  
    title1 = r"Eigenvectors"
    names_axes = (r"$\left|L\right>$", r"$\psi$") 
    
    ax1.bar(np.linspace(1,2*N-1,N)-0.42, vecs[:,0])
    ax1.bar(np.linspace(1,2*N-1,N)+0.42, vecs[:,1])
    ax1.set_title(title1)
    ax1.set_xlabel(names_axes[0])
    ax1.set_ylabel(names_axes[1])
    ax1.set_xticks(np.linspace(1,2*N-1,N))
    ax1.set_ylim((-1,1))
    
    
    eigvals= []
    thetarange = np.linspace(0, 2*pi, numtheta)
    for theta in thetarange:
        vals, troep = calc_eigs_point(N, k, lab0, theta)
        eigvals.append(vals)
    
    eigvals = np.array(eigvals)
    
    names_axes2 = (r"$\theta$ (units of $\pi$)", r"$E$") 
    title2 = r"Eigenvalues"
    for i in range(len(vals)):
        ax2.plot(thetarange/pi, eigvals.transpose()[i])
        
    ax2.plot([t, t], [-1,eigvals.max()*1.1],c='black', linestyle='--')
    ax2.set_ylim((-1,eigvals.max()*1.1))
    ax2.set_xlabel(names_axes2[0])
    ax2.set_ylabel(names_axes2[1])
    ax2.set_title(title2)


from ipywidgets import interactive, interact,fixed, FloatSlider, IntSlider
interactive_plot = interact(plot_eigvector, N=IntSlider(min=3, max=10, step=1, continuous_update=False),
                            p=IntSlider(min=-10, max=10, step=1, continuous_update=False),
                            q=IntSlider(min=3,max=10,step=1, continuous_update=False), 
                            t=FloatSlider(min=0,max=2,step=0.01,continuous_update=False))

interactive(children=(IntSlider(value=3, continuous_update=False, description='N', max=10, min=3), IntSlider(value=0, continuous_update=False, description='p', max=10, min=-10), IntSlider(value=3, continuous_update=False, description='q', max=10, min=3), FloatSlider(value=0.0, continuous_update=False, description='t', max=2.0, step=0.01), Output()), _dom_classes=('widget-interact',))